In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/Users/ashishthomaschempolil/.local/share/virtualenvs/llm-zoomcamp-By3Nq6--/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
user_question = "I just discovered the course. Can I still join it?"

# Encode the user question
user_question_embedding = embedding_model.encode(user_question)

user_question_embedding

array([ 7.82226175e-02, -4.01311480e-02,  3.86136174e-02, -1.78979419e-04,
        8.92346054e-02, -5.04591353e-02, -1.05027072e-02,  3.71056125e-02,
       -4.18713838e-02,  3.48085277e-02, -1.20701995e-02, -2.36942172e-02,
        3.87899689e-02,  1.60988104e-02,  3.50747332e-02,  3.04757687e-03,
        5.79672195e-02, -4.10627648e-02, -3.41552906e-02, -2.56396793e-02,
       -3.55264321e-02,  1.42908283e-02, -1.62800662e-02,  3.21446136e-02,
       -4.66897637e-02,  7.89186060e-02,  4.90160808e-02,  1.56760961e-02,
       -1.69110000e-02,  2.26482209e-02,  5.60205802e-02, -3.98361497e-02,
        6.77409470e-02, -1.20210163e-02,  1.12619717e-03, -1.94394737e-02,
       -2.65951362e-02,  1.06178075e-02,  1.69687793e-02,  1.13488315e-02,
       -2.97062900e-02,  5.25258556e-02, -1.41453119e-02,  4.61699925e-02,
        1.17066409e-02, -2.38052960e-02, -6.32557943e-02, -1.92041676e-02,
       -7.10593350e-03,  3.24167535e-02,  2.49618571e-02, -5.27498079e-03,
        2.01149583e-02, -

# Prepate the documents

In [7]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

course = "machine-learning-zoomcamp"
documents = [doc for doc in documents if doc['course'] == course]

len(documents) == 375

True

In [8]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [10]:
embeddings = []

for doc in documents:
    question = doc['question']
    text = doc['text']
    qa_text = f"{question} {text}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

import numpy as np

X = np.array(embeddings)

X.shape

(375, 768)

# Search

In [11]:
user_question_embedding.shape

(768,)

In [16]:
v = user_question_embedding

scores = X.dot(v)

#highest score

idx = np.argmax(scores)

scores[idx]

0.65065753

# Vector Search

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

# Hit Rate

In [18]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [23]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [57]:
# Hit rate: Defined Measures the proportion of queries for which at least one relevant document is retrieved in the top k results.
# Formula: HR@k = (Number of queries with at least one relevant document in top k) / |Q|

def hit_rate(ground_truth, search_engine, k=5, **kwargs):
    num_queries = len(ground_truth)
    num_hits = 0

    for gt in ground_truth:
        gt_q = gt['question']
        gt_doc_id = gt['document']
        v = embedding_model.encode(gt_q)
        search_results = search_engine.search(v, num_results=k, **kwargs)
        search_results_ids = [r['id'] for r in search_results]

        #check if the document_id is in the search results
        if gt_doc_id in search_results_ids:
            num_hits += 1

    return num_hits / num_queries

In [29]:
hit_rate(ground_truth, search_engine, k=5)

0.9398907103825137

# Elastic Search

In [31]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '88dae1711f88', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5iv8RXAgTZOf6VQjtfOfZw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [48]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"}, # document id
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = 'qa_index'

#delete the index
es_client.indices.delete(index=index_name, ignore=[400, 404])
# create the index
es_client.indices.create(index=index_name, body=index_settings)

/var/folders/wh/h8m35s8x51nb01cj6wl7489m0000gn/T/ipykernel_11481/1789283777.py:21: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=index_name, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'qa_index'})

In [49]:
for idx, doc in enumerate(documents):
    try:
        vector = embeddings[idx]
        doc['text_vector'] = vector
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [50]:
print(user_question)

I just discovered the course. Can I still join it?


In [51]:
user_question_embedding[0]

0.07822262

In [65]:
knn_query = {
    "field": "text_vector",
    "query_vector": user_question_embedding,
    "k": 5,
    "num_candidates": len(documents)
}

res = es_client.search(index=index_name, knn=knn_query, source=["text", "section", "question", "course", 'id'])

In [66]:
res['hits']['hits'][0]['_source']

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

In [67]:
#Elasticsearch uses approximate techniques to make it faster.

# Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

# What's hitrate for our dataset for Elastic?

def hit_rate_elastic(ground_truth, es_client, index_name, k=5):
    num_queries = len(ground_truth)
    num_hits = 0

    for gt in ground_truth:
        gt_q = gt['question']
        gt_doc_id = gt['document']
        v = embedding_model.encode(gt_q)

        knn_query = {
            "field": "text_vector",
            "query_vector": v,
            "k": k,
            "num_candidates": len(documents)
        }

        res = es_client.search(index=index_name, knn=knn_query, source=["id"])
        search_results_ids = [r['_source']['id'] for r in res['hits']['hits']]

        if gt_doc_id in search_results_ids:
            num_hits += 1

    return num_hits / num_queries

In [68]:
hit_rate_elastic(ground_truth, es_client, index_name, k=5)

0.9398907103825137